## マルチエージェント: AutoGen サンプル (日本語版)
<!--
TRANSLATION_METADATA:
{
  "source_file": "08-multi-agent/code_samples/08-autogen.ipynb",
  "language": "ja",
  "translated_at": "2025-08-28T00:00:00Z",
  "translator": "human-reviewed"
}
-->
英語版 `08-autogen.ipynb` の翻訳です。動作ロジックは同一で、システムメッセージのみ日本語化し可読性を高めています。

目的: AutoGen の RoundRobinGroupChat と単純なテキスト終端条件を使った最小グループチャット(プランナー→コンシェルジュ承認)の流れを示す。

In [1]:
import os
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_agentchat.base import TaskResult
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

### モデルクライアント初期化

In [2]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # GitHub の PAT を環境変数 GITHUB_TOKEN として設定しておく必要があります。
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

c:\VSCode - Work\ai-agents-for-beginners\ai-agents-for-beginners\myenv\Lib\site-packages\autogen_ext\models\azure\_azure_ai_client.py:307: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(config["model_info"])


### エージェント定義

In [3]:
frontdesk_agent = AssistantAgent(
    "planner_agent",
    model_client=client,
    description="旅行計画を支援するアシスタント",
    system_message="""
    あなたは 10 年の経験を持つフロントデスクの旅行プランナーです。多くの顧客を捌くため簡潔さで知られています。
    目的: 旅行者に最適なアクティビティと訪問場所を 1 件ずつ推奨すること。
    1 回の応答では必ず 1 つの推奨のみを提示してください。
    雑談は避け、与えられたゴールに集中します。
    他エージェントからの改善提案は次の回答で考慮してください。
    """,
)
concierge_agent = AssistantAgent(
    "concierge_agent",
    model_client=client,
    description="ローカル体験を重視するコンシェルジュ",
    system_message="""
    あなたはホテルのコンシェルジュで、旅行者に最もローカルで本物の体験を提供することに強い意見を持っています。
    フロントデスクの提案が観光地に偏らず最良なら 'APPROVE' とだけ応答してください。
    改善余地がある場合は、具体例を挙げず抽象的に改善指針を述べてください。
    """,
)

### 実行 (APPROVE が出るまでラウンドロビン)

In [7]:
# === 実行部 =============================================================
# TextMentionTermination: 指定した文字列('APPROVE')がメッセージ内に現れたら終了条件を満たす
termination = TextMentionTermination("APPROVE")

# RoundRobinGroupChat: frontdesk -> concierge -> frontdesk ... の順で順番に発話させるチーム
team = RoundRobinGroupChat(
    [frontdesk_agent, concierge_agent],
    termination_condition=termination,
)

# 可読性向上: 各メッセージを整形表示し最後に全文を表示する（トークン集計は省略）
conversation_log = []

def _summarize(message, turn):
    role = getattr(message, "source", getattr(message, "role", "unknown"))
    content = getattr(message, "content", str(message))
    return {"turn": turn, "role": role, "content": content}

def _truncate(txt, max_len=80):
    return txt if len(txt) <= max_len else txt[:max_len - 3] + "..."

turn = 0
async for message in team.run_stream(task="パリ旅行の計画を立てたいです。おすすめを提案してください。"):
    if isinstance(message, TaskResult):
        print(f"\n[END] Stop Reason: {message.stop_reason}")
    else:
        turn += 1
        info = _summarize(message, turn)
        conversation_log.append(info)
        print(f"[{turn:02d}] {info['role']:<12} | { _truncate(info['content']) }")

# --- フルログ出力 -------------------------------------------------------
if conversation_log:
    print("\n=== Conversation Transcript ===")
    for entry in conversation_log:
        print(f"\n[Turn {entry['turn']:02d}] Role: {entry['role']}\n{entry['content']}")
# =====================================================================

[01] user         | パリ旅行の計画を立てたいです。おすすめを提案してください。
[02] planner_agent | パリのピガール地区を訪れてみてください。ここは伝統的な雰囲気が残るエリアで、地元のバルやマーケットが楽しめます。また、モンマルトルの丘へも近いため、アート...
[02] planner_agent | パリのピガール地区を訪れてみてください。ここは伝統的な雰囲気が残るエリアで、地元のバルやマーケットが楽しめます。また、モンマルトルの丘へも近いため、アート...
[03] concierge_agent | 改善の余地があります。観光名所に捉われず、地元の人々が普段利用するスポットや独自の文化体験を強調してみてください。
[03] concierge_agent | 改善の余地があります。観光名所に捉われず、地元の人々が普段利用するスポットや独自の文化体験を強調してみてください。
[04] planner_agent | パリのバスティーユ市場をおすすめします。ここは地元の人々が新鮮な食材や特産品を購入する場所で、活気に満ちた雰囲気があります。市場ではフランスの食文化を体験...
[04] planner_agent | パリのバスティーユ市場をおすすめします。ここは地元の人々が新鮮な食材や特産品を購入する場所で、活気に満ちた雰囲気があります。市場ではフランスの食文化を体験...
[05] concierge_agent | APPROVE

[END] Stop Reason: Text 'APPROVE' mentioned

=== Conversation Transcript ===

[Turn 01] Role: user
パリ旅行の計画を立てたいです。おすすめを提案してください。

[Turn 02] Role: planner_agent
パリのピガール地区を訪れてみてください。ここは伝統的な雰囲気が残るエリアで、地元のバルやマーケットが楽しめます。また、モンマルトルの丘へも近いため、アートを感じながら地元の生活を体験できます。

[Turn 03] Role: concierge_agent
改善の余地があります。観光名所に捉われず、地元の人々が普段利用するスポットや独自の文化体験を強調してみてください。
